In [29]:
%pwd

'd:\\'

In [36]:
import os 
os.chdir('D:\Text_Summarizer1')
%pwd

'D:\\Text_Summarizer1'

In [37]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True) 
class DataTransformationConfig:
    ''' return type --- these variable'''  
    root_dir : Path 
    data_path : Path 
    tokenizer_name : Path  
    

In [38]:
from textSummarizer.constants import * 
from textSummarizer.utils.common import read_yaml, create_directories 

In [39]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH): 
        
        print(config_filepath)
        self.config = read_yaml((config_filepath)) 
        self.params = read_yaml((params_filepath))

        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir]) 

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir, 
            data_path = config.data_path,
            tokenizer_name= config.tokenizer_name
        )

        return data_transformation_config
    

In [40]:
from textSummarizer.logging import logger 
from transformers import AutoTokenizer 
from datasets import load_dataset, load_from_disk


In [41]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig): 
        self.config = config 
        # initializing the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    # coonverts the data into numerical representation [ word embedding]
    def convert_examples_to_features(self,example_batch):
        # print(example_batch)
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self): 
        dataset_samsum = load_from_disk(self.config.data_path) 
        # passing only batched data
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'samsum_dataset'))


    
    

In [42]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
    
except Exception as e:
    raise e

config\config.yaml
 in common file
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILE': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}}
[2023-10-06 10:24:30,352: INFO: common: yaml file: config\config.yaml successfully loaded]
 in common file
{'key': 'value'}
[2023-10-06 10:24:30,355: INFO: common: yaml file: params.yaml successfully loaded]
[2023-10-06 10:24:30,355: INFO: common: created directory at: artifacts]
[2023-10-06 10:24:30,355: INFO: common: created direc

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

c:\Users\jrspa\anaconda3\envs\text_S\lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]